## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import numpy as np 

A = np.random.randint(0, 1000, size=(1_000_000,))

def f1(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [ ]:
f1(A)

In [ ]:
%timeit f1(A)

In [ ]:
def f2(A):
    acc = 0
    for ai in A:
        acc += ai + 100
    return acc / len(acc)

In [ ]:
%timeit f2(A)

In [ ]:
def f3(A):
    return sum(A) / len(A) + 100

In [ ]:
%timeit f3(A)

In [ ]:
%lprun -f f1 f1(A)

In [ ]:
import numba

In [ ]:
@numba.njit
def f5(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [ ]:
f5(A)

In [ ]:
%timeit f5(A)

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [ ]:
def g1(df):
#     letters = ['a', 'b', 'c', 'd', 'e']
    let = ['f','g']
    dfs = []
    for letter in let:
        q = df[df['key']!=let]
#         q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [ ]:
return df[df['key'].isin(['a', 'b', 'c', 'd', 'e'])]

In [ ]:
dfs

In [ ]:
%timeit g1(df)

In [ ]:
%timeit g(df)

In [ ]:
%lprun -f g g(df)

## Лабораторная работа 3

In [7]:
!pip install line_profiler

In [8]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
import pandas as pd

In [21]:
recipes = pd.read_csv('./data/recipes_sample.csv', sep=',', parse_dates=['submitted'])
reviews = pd.read_csv('./data/reviews_sample.csv', sep=',', index_col=0, parse_dates=['date'])

In [22]:
#A 

def A():
    c = 0
    a = 0
    for index, row in reviews.iterrows():
        if row["date"].year == 2010:
            a += row["rating"]
            c += 1

    return a/c

res_A = A()
res_A

4.4544402182900615

In [5]:
%timeit A()

17.5 s ± 775 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# B

def B():
    c = 0
    a = 0
    y = reviews[reviews['date'].dt.year == 2010]
    for index, row in y.iterrows():
        a += row["rating"]
        c += 1

    return a/c
    
res_B = B()
res_B

4.4544402182900615

In [7]:
%timeit B()

2.42 s ± 403 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# C

def C():
    y = reviews['date'].dt.year == 2010
    return reviews.loc[y, 'rating'].mean()

rC = C()
rC

4.4544402182900615

In [11]:
%timeit C()

13.7 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [14]:
# A самая медленная ф-ция из-за цикла for
%lprun -f A A()

Timer unit: 1e-06 s

Total time: 67.3542 s
File: <ipython-input-4-7f82946fe955>
Function: A at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def A():
     4         1          3.0      3.0      0.0      c = 0
     5         1          2.0      2.0      0.0      a = 0
     6    126697   55169446.0    435.4     81.9      for index, row in reviews.iterrows():
     7    126696   11426662.0     90.2     17.0          if row["date"].year == 2010:
     8     12094     739045.0     61.1      1.1              a += row["rating"]
     9     12094      19087.0      1.6      0.0              c += 1
    10                                           
    11         1          1.0      1.0      0.0      return a/c

In [15]:
# Ср по скорости выполнения, снова мешает цикл for
%lprun -f B B()

Timer unit: 1e-06 s

Total time: 5.81583 s
File: <ipython-input-6-698a6b451a95>
Function: B at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def B():
     4         1          2.0      2.0      0.0      c = 0
     5         1          1.0      1.0      0.0      a = 0
     6         1      18222.0  18222.0      0.3      y = reviews[reviews['date'].dt.year == 2010]
     7     12095    4732591.0    391.3     81.4      for index, row in y.iterrows():
     8     12094    1048516.0     86.7     18.0          a += row["rating"]
     9     12094      16501.0      1.4      0.3          c += 1
    10                                           
    11         1          2.0      2.0      0.0      return a/c

In [16]:
# Самая быстрая функция
%lprun -f C C()

Timer unit: 1e-06 s

Total time: 0.014732 s
File: <ipython-input-10-479d98933cba>
Function: C at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def C():
     4         1      13101.0  13101.0     88.9      y = reviews['date'].dt.year == 2010
     5         1       1631.0   1631.0     11.1      return reviews.loc[y, 'rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [25]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [24]:
%timeit get_word_reviews_count(reviews)

30.9 s ± 706 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

In [ ]:
# На цикл for (3 и 12 строки кода) тратится больше всего времени 
# + if (7 строка кода) 
# + определение длины слов (16 строка кода)

In [26]:
def optim(df):
    word_reviews = {}
    for i, row in df.dropna(subset=['review']).iterrows():
        for word in row['review'].split(' '):
            if word not in word_reviews:
                word_reviews[word] = 0
            word_reviews[word] += 1
    return word_reviews

In [29]:
%timeit optim(reviews)

15.5 s ± 2.23 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%lprun -f optim optim(reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
reviews_dt = pd.DataFrame(reviews)

recipe_groupsA - A

reviews_data_frame - reviews_dt

recipe_groupsFs - F

result_for_calc - res

In [ ]:
A = reviews_dt[reviews_dt['rating']>0].groupby(by='recipe_id').mean().reset_index() #A в формуле
F = reviews_dt[reviews_dt['rating']>0].groupby(by='recipe_id')['rating'].agg(list) #F в формуле
A['F'] = A['recipe_id'].map(F)
res = A.drop(['user_id'], axis = 1)
means = res['rating'].to_list()
f_lst = res['F'].to_list()
res

In [ ]:
import itertools
def mape_A(rawA, rawF):
    result = []
    for (avg, items) in zip(rawA, rawF):
        summa = int(0)
        for item in items:
            summa += abs( avg - item )/ avg
        summa = summa * 100 / len(items)
        result.append(summa)
    
    return result
mape_A(res['rating'], res['F'])[:10]

In [ ]:
%timeit mape_A(means, fs)

In [ ]:
import numba
from numba.typed import List

@numba.njit
def mape_B(rawA, rawF):
    
    result = List()
    for (avg, items) in zip(rawA, rawF):
        summa = int(0)
        for item in items:
            summa += abs( avg - item )/ avg
        summa = summa * 100 / len(items)
        result.append(summa)
    
    return result

means_new = List()
[means_new.append(x) for x in means]
fs_new = List()
[fs_new.append(List(i for i in x)) for x in fs]
mape_B(means_new, fs_new)[:10]

In [ ]:
%timeit mape_B(means_new, fs_new)

In [ ]:
def mape_C(rawA, rawF):
    result = np.array([])
    for (avg, items) in zip(rawA, rawF):
        summa = int(0)
        for item in items:
            summa += np.fabs( avg - item )/ avg
        summa = summa * 100 / len(items)
        result = np.append(result, summa)
    return result
#n - кол-во строк
mape_C(res['rating'], res['F'])[:10]

In [ ]:
%timeit mape_C(means, fs)

In [ ]:
@numba.njit
def mape_D(rawA, rawF):   
    result = List()
    for (avg, items) in zip(rawA, rawF):
        summa = int(0)
        for item in items:
            summa += np.fabs( avg - item )/ avg
        summa = summa * 100 / len(items)
        result.append(summa)
    
    return result

means_new = List()
[means_new.append(x) for x in means]
fs_new = List()
[fs_new.append(List(i for i in x)) for x in fs]
mape_D(means_new, fs_new)[:10]

In [ ]:
%timeit mape_D(means_new, fs_new)

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4

In [4]:
import numpy as np 

In [17]:
def A(n, a):
    k = 0
    mx = max(a)
    while True:
        broken = False
        k += 1
        if a.count(a[0]) == len(a):
#         if len(set(a)) == 1:
            return(k-1)
            broken = True
            break
        for i in range(n - 1):
            if a[i] <= a[i + 1] and a[i] < mx:
                a[i] += 1
            elif len(set(a)) == 1:
                return(k)
                broken = True
                break
            elif a[i] > a[i + 1]:
                return(-1)
                broken = True
                break
        if broken:
            break


n = int(input())
a = input().split(' ')

a = [int(item) for item in a]

A(n, a)

2
1 2


1

In [ ]:
while True:
    broken = False
    k += 1
    if a.count(a[0]) == len(a):
#         if len(set(a)) == 1:
        return print(k-1)
        broken = True
        break
    for i in range(n - 1):
        if a[i] <= a[i + 1] and a[i] < mx:
            a[i] += 1
        elif len(set(a)) == 1:
            return print(k)
            broken = True
            break
        elif a[i] > a[i + 1]:
            return print(-1)
            broken = True
            break
    if broken:
        break

In [18]:
%lprun -f A A(n, a)

In [40]:
def A():    
    n = input()
    a = input()

    a = [int(item) for item in a.split(' ')]

    k = 0
    mx = max(a)
    while True:
        broken = False
        k += 1
        if a.count(a[0]) == len(a):
            print(k-1)
            broken = True
            break
        for i in range(int(n) - 1):
            if a[i] <= a[i + 1] and a[i] < mx:
                a[i] += 1
            elif len(set(a)) == 1:
                print(k)
                broken = True
                break
            elif a[i] > a[i + 1]:
                print(-1)
                broken = True
                break
        if broken:
            break

In [41]:
%lprun -f A A()

2
1 2
1


In [42]:
A()

2
1 2
1


In [ ]:
n = input()
a = input()

a = [int(item) for item in a.split(' ')]

k = 0
mx = max(a)
while True:
    broken = False
    k += 1
    if a.count(a[0]) == len(a):
        print(k-1)
        broken = True
        break
    for i in range(int(n) - 1):
        if a[i] <= a[i + 1] and a[i] < mx:
            a[i] += 1
        elif len(set(a)) == 1:
            print(k)
            broken = True
            break
        elif a[i] > a[i + 1]:
            print(-1)
            broken = True
            break
    if broken:
        break

# 

In [49]:
def B():
#     with open('input.txt', 'r') as f:
    f = open("input.txt", 'r')
    nums = f.read().splitlines()
    n = nums[0]
    a = [int(item) for item in list(nums[1].split(' '))] 

    #a = [int(item) for item in a]

    k = 0
    mx = max(a)
    while True:
        broken = False
        k += 1
        if a.count(a[0]) == len(a):
            print(k-1)
            broken = True
            break
        for i in range(int(n) - 1):
            if a[i] <= a[i + 1] and a[i] < mx:
                a[i] += 1
            elif len(set(a)) == 1:
                print(k)
                broken = True
                break
            elif a[i] > a[i + 1]:
                print(-1)
                broken = True
                break
        if broken:
            break

In [50]:
%lprun -f B B()

1


Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def B():
     2                                           #     with open('input.txt', 'r') as f:
     3         1        199.0    199.0     28.2      f = open("input.txt", 'r')
     4         1         32.0     32.0      4.5      nums = f.read().splitlines()
     5         1          2.0      2.0      0.3      n = nums[0]
     6         1         11.0     11.0      1.6      a = [int(item) for item in list(nums[1].split(' '))] 
     7                                           
     8                                               #a = [int(item) for item in a]
     9                                           
    10         1          1.0      1.0      0.1      k = 0
    11         1          4.0      4.0      0.6      mx = max(a)
    12                                               while True:
    13         2          4.0      2.0      0.6          broken = False
    14         2          3.0      1.5      0.4          k += 1
    15         2          5.0      2.5      0.7          if a.count(a[0]) == len(a):
    16         1        427.0    427.0     60.6              print(k-1)
    17         1          3.0      3.0      0.4              broken = True
    18         1          1.0      1.0      0.1              break
    19         2          6.0      3.0      0.9          for i in range(int(n) - 1):
    20         1          3.0      3.0      0.4              if a[i] <= a[i + 1] and a[i] < mx:
    21         1          2.0      2.0      0.3                  a[i] += 1
    22                                                       elif len(set(a)) == 1:
    23                                                           print(k)
    24                                                           broken = True
    25                                                           break
    26                                                       elif a[i] > a[i + 1]:
    27                                                           print(-1)
    28                                                           broken = True
    29                                                           break
    30         1          2.0      2.0      0.3          if broken:
    31                                                       break